## First import some necessary packages

In [1]:
import logging
import pathlib

import colorcet as cc
import dask.dataframe as dd
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import hvplot.xarray  # noqa: API import
import ipyleaflet
import numpy as np
import pandas as pd
import panel as pn

from random import shuffle, uniform

import IPython
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from ipywidgets import interact
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator

# Activate Panel extension to make interactive visualizations
pn.extension()

C:\Users\khdeb\miniforge-pypy3\envs\coastal\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 10.0.1. Please consider upgrading.
  warnings.warn(


In [2]:
# # Read questions from cloud storage

# import coastal_dynamics as cd

# questions = cd.read_questions(
#     "az://coastal-dynamics/questions/5_cross_shore_transport_hashed.json",
#     storage_options={"account_name": "coclico"},
# )

# (Cross-shore) sediment transport
Welcome to the notebook of week 5! The main topic of this notebook is beach states (section 7.3 in the book). These are discussed also in the context of coastal classification (chapter 2 in the book). This notebook consists of the following sections:
* Coastal classification
* Beach states

Each section contains questions for you to practice with (cross-shore) sediment transport. Let's get started!

## Coastal classification
As you know from chapter 2 of the book and the week 1 notebook, coasts can be classified based on tectonics and processes. In this notebook we will look at different Brazilian beaches, and try to classify them. Let's start with loading the dataset with data for Brazilian beaches. This dataset contains the significant wave height (Hs), the peak period (T), the mean tidal range (MTR), and the coastal slope for different coordinates along the Brazilian coast.(adapted from https://zenodo.org/records/8200200). Specifically, we will take a look at the Brazilian province Santa Catarina. This coastal province can be subdivided in five different regions. For simplicity, we will work with the mean grain size of sediment (D) for each of these regions, which is also included in the dataset.

### Load data

In [3]:
fpath = pathlib.Path("C:/Users\khdeb\Documents\Mijn Documenten\Werk\TA Coastal and Estuarine Systems\Content development/actual development\week 5 Hs en MTR map/brazilian_coast.gpkg")

# We load this file as a GeoDataFrame, which comes with a column containing the geometry of each entry. For this dataset, these are points (longitude, latitude)
gdf = gpd.read_file(fpath)
display(gdf.head())

# Let's also load in some beaches, and their locations
beaches = (
    # ("name", "latitude", "longitude"),
    ("Campo Bom", -28.7238889, -049.0613889),
    ("Laguna", -28.4813889, -048.7650000),
    ("Enseada da Pinheira", -27.8658333, -048.6019444),
    ("Praia do Moçambique", -27.5313889, -048.4208333),
    ("Tijucas", -27.233117536346906, -48.61195557330283),
    ("Balneário Camboriú", -26.99587680030832, -48.623795816849324),
    ("Do Ubatuba", -26.197513121303377, -48.526129178088894),
    ("Barra Velha", -26.586511049944388, -48.667347851199345)
)

beaches_df = pd.DataFrame(data=beaches, columns=['name', 'lat', 'lon'])
beaches_df.head()

,Region,Hs [m],MTR [m],T [s],D [mm],nearshore slope [degrees],geometry
0,Region 1,0.78,1.462,7.95,0.3,0.154125,POINT (-48.60344 -26.00940)
1,Region 1,0.78,1.458,7.95,0.3,0.446898,POINT (-48.60532 -26.01826)
2,Region 1,0.78,1.454,7.96,0.3,0.582678,POINT (-48.60689 -26.02717)
3,Region 1,0.78,1.451,7.96,0.3,0.544294,POINT (-48.60809 -26.03613)
4,Region 1,0.78,1.447,7.96,0.3,0.605021,POINT (-48.60859 -26.04514)


,name,lat,lon
0,Campo Bom,-28.723889,-49.061389
1,Laguna,-28.481389,-48.765000
2,Enseada da Pinheira,-27.865833,-48.601944
3,Praia do Moçambique,-27.531389,-48.420833
4,Tijucas,-27.233118,-48.611956


### Define plot function

**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made! !

In [4]:
def plot_brazilian_coast(plot_where='pop-out'):
    """
    change value of 'plot_where' to:
    'inline' if you would like the plot to show in the notebook
    'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)
    """
    
    # below we build the widget
    title_bar = pn.Row(
        pn.pane.Markdown("## Brazilian beaches",
                         styles={"color": "black"},
                         sizing_mode="fixed",
                         margin=(10,5,10,15),
        ),
        align='center',
    )

    # dropdown menu of beaches
    options = beaches_df.name.to_list()
    beaches_dropdown = pn.widgets.Select(
        name="Beach", options=options, value=np.random.choice(options))

    @pn.depends(beaches_dropdown.param.value)
    def plot_beaches(name, plot_size=0.04):
        
        beach = beaches_df.loc[beaches_df['name']==name].copy()
        lat, lon = beach[['lat', 'lon']].values.flatten()
        lat, lon = np.float64(lat), np.float64(lon)
    
        points = gdf.hvplot.points(
            geo=True,
            tiles='ESRI',
            ylabel='Latitude [deg]',
            xlabel='Longitude [deg]',
            xlim=(lon-plot_size/2, lon+plot_size/2),
            ylim=(lat-plot_size/2, lat+plot_size/2),
            tools=['tap'],
            hover_cols=["Longitude", "Latitude", "Region", "Hs [m]", "MTR [m]", "T [s]", "D [mm]", "nearshore slope [degrees]"],
            c='Region',
            cmap='Accent',
            line_color='black',
            size=300,
            )
    
        plot = (points).opts(
            width=800,
            height=800,
            tools=['wheel_zoom']
            )
        return plot

    app = pn.Column(pn.Row(title_bar, align="center"),
                    pn.Row(beaches_dropdown, align="center"),
                    pn.Row(plot_beaches, align="center"))

    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")

### Now plot the coastal data

Execute the cell below to generate the plot by using the function we defined above. Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by the adjusting the logging warning level. 

In [5]:
logging.getLogger().setLevel(logging.ERROR)

plot_brazilian_coast(plot_where="pop-out")

Launching server at http://localhost:62114


Have a look at section 4.4.2 from the book, and figure 4.13. Let's try to classify some of the beaches using the mean tidal range and mean wave height! \
\
**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made!

In [6]:
# Replot fig 4.13

Let's plot some of the brazilian beaches and see how they are classified using the figure. Use the brazilian beaches plot to determine relevant values for the mean tidal range and the mean wave height.

In [7]:
# Let students calculate relevant parameter

In [8]:
# Replot with relevant parameter

Using this figure, and the brazilian beaches plot, answer the questions below.

In [9]:
# # Start with wave-dominated beaches
# - features of beaches they see in the plot
# - compare two beaches
# - ask reflective questions about why some beach with higher tidal range can still be wave dominated
# - ask about barrier islands
# - ask about value for some beach, and whether they expect it to be accurate
# - ask if they see vegetation, and what this indicates w.r.t. dominant process
# - 

This is the end of the first part of this notebook. You can continue with the next part on beach states!

## Beach states
From section 7.3 from the book, you know that a wave-dominated beach may be classified as reflective or dissipative, or an intermediate state in between. A common classifier used besides the Iribarren number is the dimensionless fall velocity, which is calculated as follows: \
$$
\Omega = \frac{H_b}{w_s T}
$$
where $H_b$ is the wave height at breaking, $T$ is the wave period and $w_s$ is the fall velocity. \
\
With data on wave height at breaking being limited, let's be pragmatic and use the beach slope as a proxy for classifying a beach as being dissipative, intermidate, or reflective. We can get the beach slope data from the same dataset as used above.

Finish the code below with some of the beaches to show where they would lie in figure 7.10.

In [12]:
data = (
    # ("label", slope [degrees], mean diameter [mm]),
    ("Campo Bom", 2.987, 0.21),
)

df = pd.DataFrame(data=data, columns=['label', 'slope [degrees]', 'D [mm]'])
df.head()

,label,slope [degrees],D [mm]
0,Campo Bom,2.987,0.21


In [13]:
points = df.hvplot.points(x='slope [degrees]', y='D [mm]', by='label')

hlines = hv.HLines([0.25, 0.5]).opts(color='lightgrey')
vlines = hv.VLines([3.5, 8.5]).opts(color='lightgrey')

classify_labels = hv.Text(1.75, 0.95, "dissipative", fontsize=10) * \
                  hv.Text(6, 0.95, "intermediate", fontsize=10) * \
                  hv.Text(13, 0.95, "reflective", fontsize=10)

grain_labels = hv.Text(15.8, 0.75, "coarse sand", fontsize=10, halign='right') * \
               hv.Text(15.8, 0.375, "medium sand", fontsize=10, halign='right') * \
               hv.Text(15.8, 0.125, "fine sand", fontsize=10, halign='right')


(
    points * hlines * vlines * classify_labels * grain_labels
).opts(ylim=(0, 1), xlim=(0, 16))

:Overlay
   .NdOverlay.I :NdOverlay   [label]
      :Points   [slope [degrees],D [mm]]
   .HLines.I    :HLines   [y]
   .VLines.I    :VLines   [x]
   .Text.I      :Text   [x,y]
   .Text.II     :Text   [x,y]
   .Text.III    :Text   [x,y]
   .Text.IV     :Text   [x,y]
   .Text.V      :Text   [x,y]
   .Text.VI     :Text   [x,y]